# Neural Networks - A Practical Introduction
by _Minho Menezes_  

---

## Multilayer Perceptron for Computer Vision



---

### Libraries

In [1]:
## LIBRARIES ##
import numpy as np                         # Library for Numerical and Matricial Operations
import matplotlib.pyplot as plt            # Library for Generating Visualizations
import pandas as pd                        # Library for Handling Datasets
from tools.tools import Tools as tl        # Library for some Utilitary Tools

# Function for loading the MNIST dataset into a Numpy Matrix
import pickle

def loadMNIST():
    with open("data/mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"].T, mnist["test_images"].T, mnist["training_labels"].T, mnist["test_labels"].T

### The Neural Networks Class

In [2]:
## CLASS: Multilayer Perceptron ##
class MultilayerPerceptron:
    
    # CLASS CONSTRUCTOR
    def __init__(self, n_neurons=[2, 5, 1]):
        if(len(n_neurons) < 2):
            raise ValueError("The network must have at least two layers! (The input and the output layers)")
        
        # Network Architecture
        self.hidden_layers = len(n_neurons)-2
        self.n_neurons = n_neurons
        self.W = []
        
        # Adjusting the Network architecture
        for i in range(1, len(n_neurons)):
            self.W.append( np.random.randn(self.n_neurons[i-1]+1 , self.n_neurons[i]) )
        
    # ACTIVATION FUNCTION
    def activate(self,Z):
        return 1 / (1 + np.exp(-Z))
    
    # FORWARD PROPAGATION
    def forward(self, X):
        # Activation List
        A = []
        
        # Input Layer Activation
        A.append( np.vstack([np.ones([1, X.shape[1]]), X]) )
        
        # Hidden Layer Activation
        for i in range(0, self.hidden_layers):
            Z = np.matmul(self.W[i].T, A[-1])
            Z = self.activate(Z)
            
            A.append( np.vstack([np.ones([1, Z.shape[1]]), Z]) )
        
        # Output Layer Activation
        Z = np.matmul(self.W[-1].T, A[-1])
        Z = self.activate(Z)

        A.append(Z)
        
        return A
    
    # CLASSIFICATION PREDICTION
    def predict(self, X):
        A = self.forward(X)
        
        if(self.n_neurons[-1] > 1):
            return A[-1].argmax(axis=0)
        else:
            return (A[-1] > 0.5).astype(int)
    
    # LOSS FUNCTION
    def loss(self, y, y_hat):
        m = y.shape[1]
        return -(1/m) * np.sum(y * np.log(y_hat) + (1-y) * np.log(1 - y_hat))
    
    # ACCURACY FUNCTION
    def accuracy(self, y, y_hat):
        m = y.shape[1]
        return (1/m) * np.sum(y == y_hat) * 100
    
    # BACKPROPAGATION
    def backpropagate(self, A, y):
        # A primeira matriz de erros é calculada diretamente da diferença entre a classe real e a prevista
        E = []
        E.append( A[-1] - y )

        # O erro é, então, propagado para trás até termos os erros da primeira Camada oculta
        for i in range(self.hidden_layers, 0, -1):
            E.append( np.matmul(self.W[i], E[-1]) * A[i] * (1-A[i]) )
            E[-1] = E[-1][1:,:]

        # Retornamos o erro calculado em todas as camadas, na ordem inversa do cálculo
        return E[::-1]
    
    # GRADIENT DESCENT TRAINING
    def train(self, X_train, y_train, alpha=1e-3, maxIt=50000, tol=1e-5, verbose=False):
        m = X_train.shape[1]
    
        # Define o Histórico de Erros e algumas variáveis auxiliares
        errorHist = []
        previousLoss = 0

        # Realiza o treino por Gradiente Descendente
        for it in range(0, maxIt):
            # 1. Calculamos a ativação de todos os neurônios (Forward Propagation) e 
            #    retropropagamos o erro da predição (Backpropagation)
            A = self.forward(X_train)
            E = self.backpropagate(A, y_train)
            P = self.predict(X_train)

            # 2. Calculamos o erro MSE, a acurácia do modelo e adicionamos o resultado no histórico.
            actualLoss = self.loss(y_train, A[-1])
            actualAcc = self.accuracy(y_train, P)
            errorHist.append(actualLoss)

            # 3. Realizamos o passo do Gradiente Descendente.        
            for i in range(0, self.hidden_layers+1):
                self.W[i] = self.W[i] - (alpha/m) * np.matmul(A[i], E[i].T)

            # 4. Imprimimos o resultado do treinamento a cada 50 épocas.
            if(it % 50 == 0 and verbose): 
                print("## Iteration", it, "##")
                print("Cross-Entropy Loss: \t", actualLoss)
                print("Accuracy (Training Set): {0:.3f}%".format(actualAcc))
                print("Weights\nS -> H:\n", self.W[0], "\nH -> O:\n", self.W[1])
                print("\n")

            # 5. Verificamos uma possivel convergência do treinamento, e então encerramos o laço.
            if(abs(actualLoss - previousLoss) <= tol):
                print("!!! Convergence reached !!!")
                print("## Iteration", it, "##")
                print("Cross-Entropy Loss: \t", actualLoss)
                print("Accuracy (Training Set): {0:.3f}%".format(actualAcc))
                print("Weights\nS -> H:\n", self.W[0], "\nH -> O:\n", self.W[1])
                print("\n")
                break;

            # 6. Atualizamos as variáveis auxiliares para as próximas iterações.
            previousLoss = actualLoss

        # Fim do Treinamento
        return errorHist
        
## ---------------------------- ##

---

### 1. Image Data Extraction

When working with Computer Vision, the first thing you need, of course, is to be able to open image data and format it to 

### 2. Loading the MNIST Dataset

### 3. Training the Network

### 4. Testing the Network